# Layer Model

Follows [this section](https://geotrellis.github.io/geotrellis-workshop/docs/layer-model) of the docs.

In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.2`
import $ivy.`org.locationtech.geotrellis::geotrellis-layer:3.5.2`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`
import geotrellis.raster._
import geotrellis.vector._

In [ ]:
import geotrellis.layer.{ LayoutDefinition, SpatialKey }
import geotrellis.raster.GridExtent
import geotrellis.vector.{ Extent, Point, LineString }

val layout = LayoutDefinition(
  grid = GridExtent(Extent(0, 0, 100, 100), cols = 100, rows = 100),
  tileCols = 10, tileRows = 10
)

In [ ]:
SpatialKey(0, 9).extent(layout)
SpatialKey(0, 0).extent(layout)

In [ ]:
val mp = layout.mapTransform

In [ ]:
mp.keyToExtent(SpatialKey(0, 9))

In [ ]:
layout.mapTransform.pointToKey(Point(5, 5))

In [ ]:
layout.mapTransform.pointToKey(Point(15, 15))

In [ ]:
layout.mapTransform.keyToExtent(SpatialKey(1,8))

In [ ]:
layout.mapTransform.extentToBounds(Extent(5, 5, 15, 15))

In [ ]:
layout.mapTransform.keysForGeometry(LineString(Point(5,5), Point(25,5)))

In [ ]:
import geotrellis.layer.ZoomedLayoutScheme
import geotrellis.proj4.WebMercator

val scheme = ZoomedLayoutScheme(WebMercator, tileSize = 256)

In [ ]:
val level7 = scheme.levelForZoom(7)

val zoom7 = scheme.levelForZoom(7).layout

In [ ]:
val zoom8 = scheme.levelForZoom(8).layout

In [ ]:
val ottawaWM = Extent(-8621691, 5604373, -8336168, 5805297)

In [ ]:
val tileBounds = zoom7.mapTransform.extentToBounds(ottawaWM)

In [ ]:
val tileExtent = zoom7.mapTransform.boundsToExtent(tileBounds)

In [ ]:
// Note that this is snapping out extent to closest tile borders
tileExtent.contains(ottawaWM)
// res9: Boolean = true
ottawaWM.contains(tileExtent)
// res10: Boolean = false

In [ ]:
import geotrellis.raster._
import geotrellis.raster.geotiff._
import geotrellis.layer._

val uri1 = "https://geotrellis-workshop.s3.amazonaws.com/landsat/LC81070362015218LGN00_B3.TIF"

val rs1 = GeoTiffRasterSource(uri1)

In [ ]:
val lts = rs1.tileToLayout(zoom7)

In [ ]:
lts.keys


In [ ]:
Image(lts.read(SpatialKey(64, 51)).get.band(0).renderPng())


In [ ]:
import geotrellis.proj4._
import cats.implicits._
import cats.data._

val uri2 = "https://geotrellis-workshop.s3.amazonaws.com/landsat/LC81070352015218LGN00_B3.TIF"
val rs2 = GeoTiffRasterSource(uri2)

In [ ]:
val mrs = MosaicRasterSource(NonEmptyList(rs2, List(rs1)), WebMercator)

rs1.extent
rs2.extent

rs1.extent.reproject(rs1.crs, WebMercator).combine(rs2.extent.reproject(rs2.crs, WebMercator))

mrs.extent

In [ ]:
val extentsIntersection = 
  rs1.extent.reproject(rs1.crs, WebMercator).intersection(
      rs2.extent.reproject(rs2.crs, WebMercator)
  ).get

In [ ]:
val mraster = mrs.read(extentsIntersection).get

In [ ]:
// render
Image(mraster.tile.band(0).renderPng().bytes)

In [ ]:
import geotrellis.raster.io.geotiff._

GeoTiff(mraster, WebMercator).write("mraster.tiff")

In [ ]:
// Works only with https://github.com/locationtech/geotrellis/issues/3253
// GeoTrellis 3.5.3 is required
zoom7.mapTransform.keysForGeometry(extentsIntersection.toPolygon)

In [ ]:
val ltmrs = mrs.tileToLayout(zoom7)

In [ ]:
Image(ltmrs.read(SpatialKey(113, 50)).get.band(0).renderPng())

In [ ]:
Image(ltmrs.read(SpatialKey(114, 50)).get.band(0).renderPng())

In [ ]:
import geotrellis.layer.stitch._

val t11350 = ltmrs.read(SpatialKey(113, 50)).get.band(0)
val t11450 = ltmrs.read(SpatialKey(114, 50)).get.band(0)

t11350.dimensions
t11450.dimensions

val stitched = 
  TileLayoutStitcher
    .stitch(
      Seq(
        SpatialKey(113, 50) -> t11350,
        SpatialKey(114, 50) -> t11450,
      )
    )



In [ ]:
val stitchedTile = stitched._1

stitchedTile.dimensions
Image(stitchedTile.renderPng().bytes)